## Imports/Setup

In [2]:
%run ../00_functions/00_custom_functions.ipynb
standard_setup(extra_packages=True, verbose=False)
data_ingestion_prep(start_yearmonth='1979-01', end_yearmonth='2022-12', verbose=False)

env: ESMFMKFILE=/home/ds4114/.conda/envs/leap_test202308-3/lib/esmf.mk
Files will be outputed as: .nc
ECMWF CDS API key already installed


In [7]:
%run ../00_functions/00_co2_flux_equations.ipynb

## Collection: Prepare Gas Transfer Variables

This requires several processed datasets to create variables used for flux calculation in the HPD product.
As some processing require a lot of data to download, this is set up as an example and uses prior year's data

In [4]:
if False: #if we actually downloaded all the data, then we can use those files, otherwise use prior versions
    #SST and SS should be updated to 1959 once but just not processing and downloading all that data so using different file instead
    sss_xr = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'SSS/processed/','SSS_Met-Office-Hadley-Centre_EN422f-g10-analyses_198201-202304.nc'))        
    sst_xr = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'SST/processed/','SST_ECMWF_ERA5-monthly-reanalysis-1x1-SST_195901-202304.nc')).rename({'sst_era5':'sst'})
    #SLP and wind data - actual data is very large but we would need SLP, wind speed, and wind std
    mslp = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'SLP/processed/','SLP_ECMWF_ERA5-monthly-reanalysis-1x1-MSLP_195901-202304.nc')).mslp
    era_wind = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'Wind/processed/','Wind_ECMWF_ERA5-monthly-10m-uv-wind-1x1_195901-202212.nc')).ws10
    wind_std_xr = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'Wind/processed/','Wind_ECMWF_ERA5-monthly-10m-uv-wind-1x1_195901-202212.nc')).wind_std
else: #without downlaoding full data, we can just use prior versions
    sss_xr = xr.open_dataset(f"/data/artemis/observations/EN422_salinity/processed/sss_1x1_mon_EN422_195901-202212.nc").rename({'SSS':'sss'})
    sst_xr = xr.open_dataset(f"/data/artemis/observations/ERA5/processed/ERA5_SST_1x1_mon_1958-2022.nc")
    mslp = xr.open_dataset(f'/data/artemis/observations/ERA5/processed/ERA5_MSLP_WS10_1x1_mon_1958-2022.nc').mslp / 100 # Pascals to hPa
    era_wind  = xr.open_dataset(f'/data/artemis/observations/ERA5/processed/ERA5_MSLP_WS10_1x1_mon_1958-2022.nc').ws10
    wind_std_xr = xr.open_dataset(f'/data/artemis/observations/ERA5/processed/ERA5_WS_STD_1x1_mon_1958-2022.nc')

#we already have the following sets so can use these regardless
coast_fill_xr = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'pCO2/processed/','fCO2_NOAA-NCEI_MPI-ULB-SOM-FFN_195901-202212.nc'))
ice_raw = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'Ice/originals/','Ice_Met-Office-Hadley-Centre_HadISST_187001-202309.nc')) \
          .rename({'latitude':'ylat','longitude':'xlon'})
atmos = xr_open_dataset_custom(os.path.join(global_vars['download_folder'],'xCO2/processed/','xCO2_NOAA_pCO2-fCO2-corrected-ESRL-Mauna_195901-202212.nc'))

In [5]:
#now process a bit to get on common timeframe
coast_fill = coast_fill_xr.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth))).fco2

sss = sss_xr.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth))).sss
salt_bulk = np.where(np.isnan(coast_fill),np.nan,sss)

sst = sst_xr.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth))).sst
sst_C = np.where(np.isnan(coast_fill),np.nan,sst)

ice = ice_raw.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth))).sic
ice_frac = np.where(np.isnan(coast_fill),np.nan,ice)

mslp = mslp.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth)))
press_hPa = np.where(np.isnan(coast_fill),np.nan,mslp)
era_wind = era_wind.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth)))
wind_ms = np.where(np.isnan(coast_fill),np.nan,era_wind)
wind_std_xr = wind_std_xr.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth))).ws_std
wind_std = np.where(np.isnan(coast_fill),np.nan,wind_std_xr)

atmos = atmos.sel(time=slice(str(processed_start_yearmonth_back_in_time),str(processed_end_yearmonth)))

In [8]:
#calculate 
kw_esst = calc_kw(wind_ms, wind_std, sst_C, ice_frac) #units of cm/hr
K0blk = calc_k0_weiss1974(salt_bulk, sst_C+273.15, press_hPa/1013.25)  # units of mol/L/atm

In [9]:
ds_flux_prep = xr.Dataset({'K0'        :(['time','ylat','xlon'],K0blk),
                           'kw'        :(['time','ylat','xlon'],kw_esst),
                           'ws10'      :(['time','ylat','xlon'],wind_ms),
                           'ws10_std'  :(['time','ylat','xlon'],wind_std),
                           'sss'       :(['time','ylat','xlon'],salt_bulk),
                           'sst'       :(['time','ylat','xlon'],sst_C),
                           'ice'       :(['time','ylat','xlon'],ice_frac),          #how much ice cover
                           'slp'       :(['time','ylat','xlon'],press_hPa/1013.25), #in atms
                           'atmos_fco2':(['time','ylat','xlon'],atmos.fco2.data),
                           'mauna_fco2':(['time','ylat','xlon'],atmos.mauna_loa_fco2.data),
                           'time':(['time'],ttime_back_in_time.values),
                           'ylat':(['ylat'],coast_fill.ylat.values),
                           'xlon':(['xlon'],coast_fill.xlon.values)})

ds_flux_prep['K0'].attrs['units'] = "mol/L/atm"
ds_flux_prep['kw'].attrs['units'] = "cm/hr"
ds_flux_prep['atmos_fco2'].attrs['units'] = "mu atm"
ds_flux_prep['mauna_fco2'].attrs['units'] = "mu atm"
ds_flux_prep['sst'].attrs['units'] = "deg C (ERA5)"
ds_flux_prep['ws10'].attrs['units'] = "m/s (ERA5)"
ds_flux_prep['slp'].attrs['units'] = "atmospheres"
ds_flux_prep.attrs['more_info'] = "ERA5 for wind, MSLP, & SST, Hadley Ice; E4.2.2 for SSS" #+" created by prep_gas_transfer.ipynb"

In [138]:
#not sure best place to output this but putting with SeaFlux
output_xarray_with_date(ds_flux_prep, os.path.join(data_folder_root,'SeaFlux/processed/'), 'HPD-Flux-Prep_LDEO_ERA5-K0-kw-atmos_pco2', filetype=output_file_type, overwrite=False)

Saved HPD-Flux-Prep_LDEO_ERA5-K0-kw-atmos_pco2_195901-202212.nc to /data/artemis/workspace/ds4114/online_data/SeaFlux/processed/
